In [11]:
from requests_html import HTMLSession
from requests_html import HTML
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import csv
import re

In [12]:
def ThuThapData(link,maxNumPage,fileName):
    f = open(fileName, "a",encoding="utf-8")
    f.write("Loại\tTên\tCác loại phòng\tGiá\tVị trí\tSố đánh giá\n")
    browser = webdriver.Chrome(executable_path='chromedriver.exe')
    for page in range(1,maxNumPage+1):
        url = f"{link}{page}"
        browser.get(url)
        time.sleep(3)
        r = HTML(html=browser.page_source)
        posts=r.find(".promo.is-relative")
        for stt,post in enumerate(posts):
            temp=post.text.split("\n")
            # Bỏ đi dòng khuyến mãi
            if temp[0][0]=='-':
                temp.remove(temp[0])
            # Bỏ đi trạng thái phòng
            if 'Đặt phòng ngay' in temp[1]:
                temp.remove(temp[1])
            for i in temp:
                f.write(i)
                f.write("\t")
    f.write(link)
    f.write("\t")
    f.write("\n")
    f.close()

In [13]:
#ThuThapData("https://www.luxstay.com/vi/vietnam/lam-dong/da-lat?page=",71,"dalat.csv")
#ThuThapData("https://www.luxstay.com/vi/vietnam/ha-noi?page=",93,"hanoi.csv")
#ThuThapData("https://www.luxstay.com/vi/vietnam/ho-chi-minh?page=",134,"hochiminh.csv")
#ThuThapData("https://www.luxstay.com/vi/vietnam/ba-ria-vung-tau/vung-tau?page=",31,"vungtau.csv")
#ThuThapData("https://www.luxstay.com/vi/vietnam/khanh-hoa/nha-trang?page=",34,"nhatrang.csv")
#ThuThapData("https://www.luxstay.com/vi/vietnam/quang-ninh?page=",15,"quangninh.csv")


In [14]:
def crawDataOnePage(browser,url,time_sleep,file_name):
    browser.get(url)
    browser.execute_script("window.scrollTo(0, window.scrollY + 1200)")
    time.sleep(time_sleep)
    # Tắt quảng cáo bằng ESC nếu có
    browser.find_element_by_tag_name('html').send_keys(Keys.ESCAPE)
    r = HTML(html=browser.page_source)

    f = open(file_name, "a+",encoding="utf-8")
    f.write(url)
    f.write("\t")
    # Lấy tên nhà
    try:
        f.write(r.find(".col-xs-12.col-sm-10")[0].text)
    except:
        pass
    f.write("\t")
    # Lấy loại nhà
    try:
        f.write(r.find(".mt--12")[0].text.split(" · ")[0])
    except:
        pass
    f.write("\t")
    # Lấy Diện tích
    try:
        f.write(r.find(".mt--12")[0].text.split(" · ")[1])
    except:
        pass  
    f.write("\t")
    # Lấy các lại phòng
    try:
        f.write(r.find(".mb--0")[2].text)
    except:
        pass  
    f.write("\t")
    # Lấy giá phòng
    try:
        f.write(r.find(".fadeIn")[1].text)
    except:
        pass 
    f.write("\t")
    # Lấy ví trí
    try:
        f.write(r.find(".ml--6.bold")[0].text)
    except:
        pass 
    f.write("\t")
    #lấy tiện ích
    for i in range(10):
        try:
            time.sleep(0.1)
            x=browser.find_element_by_link_text("Xem thêm")
            x.click()
            reponse = HTML(html=browser.page_source)
            break
        except:
            pass
    try:
        f.write(reponse.find(".container.container--sm.pb--48")[0].text.replace("\n",",")) 
    except:
        pass
    f.write("\n")
    f.close()

In [15]:
def crawDataMulPage(main_link,time_sleep,file_name,startNumPage,endNumPage):
    pattern=r"^/vi/rooms/\d+"
    
    fi = open(file_name, "w",encoding="utf-8")
    fi.write("Link\tTên\tLoại nhà\tDiện tích\tCác loại phòng\tGiá\tVị trí\tTiện ích\n")
    fi.close()
    browser = webdriver.Chrome(executable_path='chromedriver.exe')
    for page in range(startNumPage,endNumPage+1):
        main_url = f"{main_link}{page}"
        print(main_url)
        browser.get(main_url)
        time.sleep(time_sleep)
        r = HTML(html=browser.page_source)
        sub_links=[]
        # lọc link cần
        for link in r.links:
            if re.match(pattern,link)!=None:
                sub_links.append(link)
        print(sub_links)
        # lấy data từng link
        for sub_link in sub_links:
            print(f"https://www.luxstay.com{sub_link}")
            crawDataOnePage(browser,f"https://www.luxstay.com{sub_link}",time_sleep,file_name)


In [32]:
#crawDataMulPage("https://www.luxstay.com/vi/vietnam/quang-ninh?page=",1,"quangninh_p1.csv",1,10)
#crawDataMulPage("https://www.luxstay.com/vi/vietnam/quang-ninh?page=",1,"quangninh_p2.csv",11,15)
# crawDataMulPage("https://www.luxstay.com/vi/vietnam/khanh-hoa/nha-trang?page=",1,"nhatrang_p1.csv",1,10)
# crawDataMulPage("https://www.luxstay.com/vi/vietnam/khanh-hoa/nha-trang?page=",1,"nhatrang_p2.csv",11,20)
# crawDataMulPage("https://www.luxstay.com/vi/vietnam/khanh-hoa/nha-trang?page=",1,"nhatrang_p3.csv",21,30)
# crawDataMulPage("https://www.luxstay.com/vi/vietnam/khanh-hoa/nha-trang?page=",1,"nhatrang_p4.csv",31,34)
# crawDataMulPage("https://www.luxstay.com/vi/vietnam/ba-ria-vung-tau/vung-tau?page=",1,"vungtau_p1.csv",1,10)
# crawDataMulPage("https://www.luxstay.com/vi/vietnam/ba-ria-vung-tau/vung-tau?page=",1,"vungtau_p2.csv",11,20)
# crawDataMulPage("https://www.luxstay.com/vi/vietnam/ba-ria-vung-tau/vung-tau?page=",1,"vungtau_p3.csv",21,31)
# for page in range(1,131,10):
#     crawDataMulPage("https://www.luxstay.com/vi/vietnam/ho-chi-minh?page=",1,f"hcm_p{int(page/10+1)}.csv",page,page+9)
# crawDataMulPage("https://www.luxstay.com/vi/vietnam/ho-chi-minh?page=",1,"hcm_p14.csv",131,134)
# for page in range(1,91,10):
#     crawDataMulPage("https://www.luxstay.com/vi/vietnam/ha-noi?page=",1,f"hanoi_p{int(page/10+1)}.csv",page,page+9)
# for page in range(1,71,10):
#     crawDataMulPage("https://www.luxstay.com/vi/vietnam/lam-dong/da-lat?page=",1,f"dalat_p{int(page/10+1)}.csv",page,page+9)

https://www.luxstay.com/vi/vietnam/ba-ria-vung-tau/vung-tau?page=11
['/vi/rooms/33528', '/vi/rooms/33477', '/vi/rooms/32420', '/vi/rooms/33526', '/vi/rooms/29690', '/vi/rooms/31328', '/vi/rooms/23440', '/vi/rooms/32321', '/vi/rooms/26770', '/vi/rooms/34742', '/vi/rooms/29151', '/vi/rooms/31332', '/vi/rooms/35402', '/vi/rooms/33689', '/vi/rooms/21222', '/vi/rooms/31311', '/vi/rooms/24656', '/vi/rooms/33622', '/vi/rooms/34408', '/vi/rooms/33682']
https://www.luxstay.com/vi/rooms/33528
https://www.luxstay.com/vi/rooms/33477
https://www.luxstay.com/vi/rooms/32420
https://www.luxstay.com/vi/rooms/33526
https://www.luxstay.com/vi/rooms/29690
https://www.luxstay.com/vi/rooms/31328
https://www.luxstay.com/vi/rooms/23440
https://www.luxstay.com/vi/rooms/32321
https://www.luxstay.com/vi/rooms/26770
https://www.luxstay.com/vi/rooms/34742
https://www.luxstay.com/vi/rooms/29151
https://www.luxstay.com/vi/rooms/31332
https://www.luxstay.com/vi/rooms/35402
https://www.luxstay.com/vi/rooms/33689
https

In [23]:
def readOneFile(fileName):
    f=open(fileName,"r",encoding="utf-8")
    content=f.readlines()
    f.close()
    return content

In [33]:
# tổng hợp dữ liệu vào 1 file
f= open("data.csv","w",encoding="utf-8")
listFile=["dalat_p1","dalat_p2","dalat_p3","dalat_p4","dalat_p5","dalat_p6","dalat_p7",
          "hanoi_p1","hanoi_p2","hanoi_p3","hanoi_p4","hanoi_p5","hanoi_p6","hanoi_p7","hanoi_p8","hanoi_p9","hanoi_p10",
          "hcm_p1","hcm_p2","hcm_p3","hcm_p4","hcm_p5","hcm_p6","hcm_p7","hcm_p8","hcm_p9","hcm_p10","hcm_p11","hcm_p12","hcm_p13","hcm_p14",
          "nhatrang_p1","nhatrang_p2","nhatrang_p3","nhatrang_p4",
          "vungtau_p1","vungtau_p2","vungtau_p3",
          "quangninh_p1","quangninh_p2"]
f.write("Link\tTên\tLoại nhà\tDiện tích\tCác loại phòng\tGiá\tVị trí\tTiện ích\n")
for fileName in listFile:
    content=readOneFile(f"{fileName}.csv")
    for i in range(1,len(content)):
        f.write(content[i])
f.close()

In [39]:
import numpy as np
import pandas as pd

data_df = pd.read_csv("data.csv",sep='\t')

In [40]:
data_df.head()

,Link,Tên,Loại nhà,Diện tích,Các loại phòng,Giá,Vị trí,Tiện ích
0,https://www.luxstay.com/vi/rooms/13967,JOLIE HOUSE - BIỆT THỰ HƯỚNG RA VƯỜN - CÁCH TR...,Biệt Thự,100 m2,Nguyên căn · 5 phòng tắm · 6 giường · 4 phòng ...,"2,887,500₫ /đêm","Đà Lạt, Lâm Đồng, Vietnam",NaN
1,https://www.luxstay.com/vi/rooms/33714,Gold&Ruby House- Phòng Quadruple,Căn hộ dịch vụ,20 m2,Phòng riêng · 1 phòng tắm · 2 giường · 1 phòng...,"600,000₫ /đêm","Đà Lạt, Lâm Đồng, Vietnam","Tiện ích gia đình,Phù hợp với trẻ nhỏ,Không hú..."
2,https://www.luxstay.com/vi/rooms/15456,Maika Condotel - Triple Bedroom,Căn hộ chung cư,110 m2,Nguyên căn · 2 phòng tắm · 3 giường · 3 phòng ...,"2,750,000₫ /đêm","Đà Lạt, Lâm Đồng, Vietnam","Tiện ích gia đình,Phù hợp với trẻ nhỏ,Tiện ích..."
3,https://www.luxstay.com/vi/rooms/18878,Vườn đom đóm Đà Lạt- Phòng đơn hướng vườn rộng...,Nhà riêng,12 m2,Phòng riêng · 3 phòng tắm · 1 giường · 1 phòng...,"370,000₫ /đêm","Đà Lạt, Lâm Đồng, Vietnam","Tiện ích gia đình,Không hút thuốc,Tiện ích bếp..."
4,https://www.luxstay.com/vi/rooms/15377,Rebeka's Home 1 - Góc vườn nhỏ tĩnh lặng trước...,Nhà riêng,15 m2,Phòng riêng · 1 phòng tắm · 1 giường · 1 phòng...,"350,000₫ /đêm","Đà Lạt, Lâm Đồng, Vietnam","Tiện ích gia đình,Phù hợp với trẻ nhỏ,Tiện ích..."


In [41]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7507 entries, 0 to 7506
Data columns (total 8 columns):
Link              7507 non-null object
Tên               7489 non-null object
Loại nhà          7491 non-null object
Diện tích         7368 non-null object
Các loại phòng    7489 non-null object
Giá               7488 non-null object
Vị trí            7487 non-null object
Tiện ích          6104 non-null object
dtypes: object(8)
memory usage: 469.3+ KB
